<a href="https://colab.research.google.com/github/E-CG/AI4ENG/blob/master/03%20-%20Modelos-e-iteraciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **📦 Librerias y paquetes para la ejecución del notebook.**

In [ ]:
! pip install py7zr

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle
# Aquí debes cambiar la dirección donde tengas tus credenciales de Kaggle
! cp /content/drive/MyDrive/Modelos_I/credentials_kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download favorita-grocery-sales-forecasting
! unzip favorita-grocery-sales-forecasting.zip

In [6]:
# Librerias uso básico
import numpy as np
import pandas as pd
import math as m
import time
import py7zr
import os
from subprocess import check_output

# Librerias preprocesado
from mlxtend.preprocessing import minmax_scaling
import datetime as dt

# Librerias para gráficar
import seaborn as sns
import matplotlib.pyplot as plt

# Funciones de sklearn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression,ElasticNet,Ridge,Lasso
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error

# **🗂️Leyendo y extrayendo los archivos .csv**

In [7]:
# Ruta al archivo 7z en Google Drive
sevenzip_file_path = '/content'

# Directorio de destino para la extracción
extracted_dir = '/content/extracted_data/'

# Crear el directorio de destino si no existe
os.makedirs(extracted_dir, exist_ok=True)

# Listar los archivos .7z en el directorio de entrada
files_to_extract = [file for file in os.listdir(sevenzip_file_path) if file.endswith('.7z')]

# Iterar a través de los archivos y descomprimirlos
for file_to_extract in files_to_extract:
    with py7zr.SevenZipFile(os.path.join(sevenzip_file_path, file_to_extract), mode='r') as z:
        z.extractall(path=extracted_dir)

In [27]:
stores = pd.read_csv('/content/extracted_data/stores.csv')
items = pd.read_csv('/content/extracted_data/items.csv')
holidays_e = pd.read_csv('/content/extracted_data/holidays_events.csv', parse_dates=["date"])
transactions = pd.read_csv('/content/extracted_data/transactions.csv', parse_dates=["date"])
oil = pd.read_csv('/content/extracted_data/oil.csv', parse_dates=["date"])

# Cargar el archivo de entrenamiento en chunks
chunked_dfs = pd.read_csv("/content/extracted_data/train.csv",
                          chunksize=20000,
                          usecols=[0,1,2,3,4,5],
                          parse_dates=['date'],
                          low_memory=False)

print('Archivos cargados 🗞️✅')

Archivos cargados 🗞️✅


In [23]:
# Agrupar por estado y obtener el primer store_nbr en cada estado
tienda_por_estado = stores.groupby('state')['store_nbr'].first()
tienda_por_estado

state
Azuay                             37
Bolivar                           19
Chimborazo                        14
Cotopaxi                          12
El Oro                            40
Esmeraldas                        43
Guayas                            24
Imbabura                          15
Loja                              38
Los Rios                          31
Manabi                            52
Pastaza                           22
Pichincha                          1
Santa Elena                       25
Santo Domingo de los Tsachilas     5
Tungurahua                        23
Name: store_nbr, dtype: int64

In [30]:
# Inicializar una lista para almacenar los DataFrames filtrados
datos_filtrados = []

# Iterar a través de los chunks
for chunk in chunked_dfs:
    # Crear una máscara booleana para las tiendas seleccionadas
    mask_tiendas = chunk['store_nbr'].isin(stores['store_nbr'])
    # Crear una máscara booleana para las fechas entre 2016-01-01 a 2016-12-31
    mask_fechas = (chunk['date'] >= '2016-01-01') & (chunk['date'] <= '2016-12-31')
    # Aplicar ambas máscaras
    mask = mask_tiendas & mask_fechas

    # Aplicar la máscara y agregar las filas filtradas a la lista
    trozo_filtrado = chunk[mask]
    datos_filtrados.append(trozo_filtrado)

# Concatenar los DataFrames filtrados en uno solo
train_store_state = pd.concat(datos_filtrados, ignore_index=True)

In [31]:
train_store_state.head(3)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,66458908,2016-01-01,25,105574,12.0,False
1,66458909,2016-01-01,25,105575,9.0,False
2,66458910,2016-01-01,25,105857,3.0,False


In [32]:
# Seleccionar aleatoriamente 70.000 filas de 'train_store_state'
train_sample = train_store_state.sample(n=70000)

In [33]:
train_sample.head(3)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
30244463,96703371,2016-11-12,35,1162932,2.0,False
10371914,76830822,2016-04-20,11,1418022,7.0,False
34318696,100777604,2016-12-22,34,1388574,6.0,False


## **📄Juntando los demás archivos en train.csv y generando dataframes de prueba y entrenamiento**

Haré uso de un pipeline utilizando dos clases, una para unir los demás archivos teniendo como base el dataframe de entrenamiento 'train_store_state'

In [11]:
class MergeDataTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Constructor de la clase
        print("Inicializando MergeDataTransformer")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Unir DataFrames
        train_stores = X[0].merge(X[1], on='store_nbr')
        train_stores_oil = train_stores.merge(X[2], on='date')
        train_stores_oil_items = train_stores_oil.merge(X[3], on='item_nbr')
        train_stores_oil_items_transactions = train_stores_oil_items.merge(X[4], on=['date', 'store_nbr'])
        train_stores_oil_items_transactions_hol = train_stores_oil_items_transactions.merge(X[5], on='date')

        # Copiar DataFrame
        data = train_stores_oil_items_transactions_hol.copy(deep=True)

        # Cambiar booleanos a enteros
        data['onpromotion'] = data['onpromotion'].astype(int)
        data['transferred'] = data['transferred'].astype(int)

        # Cambiar nombres de columnas
        data.rename(columns={'type_x': 'st_type', 'type_y': 'hol_type'}, inplace=True)

        # Eliminar la columna 'id'
        data.drop(['id'], axis=1, inplace=True)

        # Mostrar las primeras filas del DataFrame resultante
        print(data.head())

        # Manipular la columna de fecha
        data['date'] = pd.to_datetime(data['date'])
        data['date'] = data['date'].map(dt.datetime.toordinal)

        return data

Divisor de Datos (SplitDataTransformer) ➗

Este transformador personalizado elimina la columna 'date' del conjunto de datos original y divide los datos en tres partes:

1. **Datos Numéricos (`data_numeric_df`):** Incluye todas las columnas numéricas del conjunto de datos original.

2. **Datos Categóricos (`data_categorical_df`):** Incluye todas las columnas categóricas del conjunto de datos original.

3. **Fecha (`data_date_df`):** Incluye la columna 'date' original.

In [12]:
class SplitDataTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Constructor de la clase
        print("Inicializando SplitDataTransformer")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Eliminar la columna 'date' del DataFrame original
        df_without_date = X.drop(['date'], axis=1)

        # Obtener columnas numéricas y categóricas
        all_columns = df_without_date.columns
        numeric_columns = df_without_date._get_numeric_data().columns
        categorical_columns = list(set(all_columns) - set(numeric_columns))

        # Crear DataFrames separados para datos numéricos, categóricos y de fecha
        data_numeric_df = X[numeric_columns]
        data_categorical_df = X[categorical_columns]
        data_date_df = X['date']

        return data_numeric_df, data_categorical_df, data_date_df

Procesamiento de Datos (ProcessData) 🛤️

Este transformador personalizado realiza el procesamiento de datos, incluyendo:

1. **Datos Numéricos (`data_num_df`):**
   - Imputa valores nulos en atributos numéricos utilizando la estrategia de la media.
   - Aplica escalado estándar a los datos numéricos.

2. **Datos Categóricos (`data_cat_df`):**
   - Utiliza un codificador one-hot para representar las variables categóricas.
   - Convierte la representación codificada a un DataFrame.

3. **Fecha (`X[2]`):**
   - Mantiene la columna de fechas sin cambios.

In [13]:
class ProcessData(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("Inicializando ProcessData")

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Datos numéricos
        # Imputar valores nulos en atributos numéricos
        imputer = SimpleImputer(strategy="mean", copy=True)
        num_imputed = imputer.fit_transform(X[0])
        data_num_df = pd.DataFrame(num_imputed, columns=X[0].columns, index=X[0].index)

        # Aplicar escalado estándar
        scaler = StandardScaler()
        num_scaled = scaler.fit_transform(data_num_df)
        data_num_df = pd.DataFrame(num_scaled, columns=X[0].columns, index=X[0].index)

        # Datos categóricos
        # One-hot encoder
        cat_encoder = OneHotEncoder(sparse=False)
        data_cat_1hot = cat_encoder.fit_transform(X[1])

        # Convertir a DataFrame con n*99, donde n es el número de filas y 99 es el número de categorías
        data_cat_df = pd.DataFrame(data_cat_1hot, columns=cat_encoder.get_feature_names_out(), index=X[1].index)

        return data_num_df, data_cat_df, X[2]

Unión de DataFrames (JoinDataFrames) 🖇️

Este transformador es útil para combinar información de diferentes conjuntos de datos antes de proceder con el análisis o modelado posterior.

In [14]:
class JoinDataFrames(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("Inicializando JoinDataFrames")

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Unir DataFrames numéricos
        data_df = X[0].join(X[1])
        data_df = data_df.join(X[2])

        return data_df

Pipeline de Procesamiento de Datos 🔩

1. **Preparar Datos (`MergeDataTransformer`):**
   - Realiza las primeras manipulaciones de los datos.

2. **Dividir Datos (`SplitDataTransformer`):**
   - Separa los datos en categorías numéricas, categóricas y de fecha.

3. **Procesar Datos (`ProcessData`):**
   - Imputa valores nulos y aplica escalado a datos numéricos.
   - Codifica variables categóricas utilizando one-hot encoding.

4. **Unir Datos (`JoinDataFrames`):**
   - Combina DataFrames numéricos en uno solo.


In [20]:
# Definir la pipeline
pipeline = Pipeline([
    ('merge_data', MergeDataTransformer()),
    ('split_data', SplitDataTransformer()),
    ('process_data', ProcessData()),
    ('join_data', JoinDataFrames())
])

Inicializando MergeDataTransformer
Inicializando SplitDataTransformer
Inicializando ProcessData
Inicializando JoinDataFrames


Aplicar Pipeline y Dividir Datos ✅

In [34]:
# Aplicar la pipeline para transformar los datos
data_df = pipeline.fit_transform([train_sample, stores, oil, items, transactions, holidays_e])

# Dividir los datos según nuestra ingeniería de características
X = data_df.drop(['unit_sales', 'transactions'], axis=1)
Y = data_df[['unit_sales', 'transactions']]

        date  store_nbr  item_nbr  unit_sales  onpromotion     city  \
0 2016-12-05         35    169028         3.0            0   Playas   
1 2016-12-05         35    514172         3.0            1   Playas   
2 2016-12-05         35    426155         2.0            0   Playas   
3 2016-12-05         11   1489895         1.0            1  Cayambe   
4 2016-12-05         11   1239843         5.0            0  Cayambe   

       state st_type  cluster  dcoilwtico          family  class  perishable  \
0     Guayas       C        3       51.72       GROCERY I   1040           0   
1     Guayas       C        3       51.72       BEVERAGES   1124           0   
2     Guayas       C        3       51.72  PREPARED FOODS   2962           1   
3  Pichincha       B        6       51.72         PRODUCE   2028           1   
4  Pichincha       B        6       51.72           DAIRY   2170           1   

   transactions    hol_type locale locale_name           description  \
0           623  Add

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
